In [42]:
import pandas as pd
import numpy as np

import tensorflow as tf
# Set Random Seeds
tf.keras.utils.set_random_seed(42)
tf.random.set_seed(42)
np.random.seed(42)
# run the Enable Deterministic Operations Function
tf.config.experimental.enable_op_determinism()

from tensorflow.keras import layers 
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras import optimizers 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from imblearn.under_sampling import RandomUnderSampler
from sklearn import set_config
set_config(transform_output = 'pandas')
pd.set_option('display.max_colwidth', 250)

In [43]:
df = pd.read_csv('Data/spooky.csv', index_col='id')
df

,text,author
id,,
id26305,"This process, however, afforded me no means of ascertaining the dimensions of my dungeon; as I might make its circuit, and return to the point whence I set out, without being aware of the fact; so perfectly uniform seemed the wall.",EAP
id17569,It never once occurred to me that the fumbling might be a mere mistake.,HPL
id11008,"In his left hand was a gold snuff box, from which, as he capered down the hill, cutting all manner of fantastic steps, he took snuff incessantly with an air of the greatest possible self satisfaction.",EAP
id27763,"How lovely is spring As we looked from Windsor Terrace on the sixteen fertile counties spread beneath, speckled by happy cottages and wealthier towns, all looked as in former years, heart cheering and fair.",MWS
id12958,"Finding nothing else, not even gold, the Superintendent abandoned his attempts; but a perplexed look occasionally steals over his countenance as he sits thinking at his desk.",HPL
...,...,...
id17718,"I could have fancied, while I looked at it, that some eminent landscape painter had built it with his brush.",EAP
id08973,The lids clenched themselves together as if in a spasm.,EAP
id05267,"Mais il faut agir that is to say, a Frenchman never faints outright.",EAP


In [44]:
df.duplicated().sum()

0

In [45]:
# Determine the sequence length of each document in the corpus by using a lambda function to split the text on the spaces
seq_length = df['text'].map(lambda x : len(x.split()))
# Determine the range of sequence lengths
range_of_lengths = max(seq_length) - min(seq_length)
print(f"Range of sequence lengths: {range_of_lengths} with maximum length = {max(seq_length)} and minimum length = {min(seq_length)}")
# Determine the average of sequence lengths
average_of_lengths = seq_length.mean()
print("Average of sequence lengths:", round(average_of_lengths,2))

Range of sequence lengths: 859 with maximum length = 861 and minimum length = 2
Average of sequence lengths: 26.73


In [46]:
df['author'].value_counts()

EAP    7900
MWS    6044
HPL    5635
Name: author, dtype: int64

In [47]:
# Use the RandomUnderSampler to balance the data based on the "author" column.
rus = RandomUnderSampler(random_state = 42)
df, _ = rus.fit_resample(df, df['author'])
df['author'].value_counts()

EAP    5635
HPL    5635
MWS    5635
Name: author, dtype: int64

In [48]:
# Change the author strings to integers using the following mapping: EAP: 0, HPL: 1, MWS: 2
## Mapping dictionary
author_mapping = {'EAP': 0, 'HPL': 1, 'MWS': 2}

## Replace author strings with integers based on the mapping
df['author'] = df['author'].map(author_mapping)

df.head()

,text,author
id,,
id22483,"He then asked me, suddenly, if I had observed any thing peculiar at the scene of the atrocity.",0
id18809,"Through the exertions of Beauvais, the matter was industriously hushed up, as far as possible; and several days had elapsed before any public emotion resulted.",0
id16322,"The cold was intense, and obliged me to wrap up closely in an overcoat.",0
id13423,"I observed that, upon her first elevation of the glass, she had seemed satisfied with a momentary inspection of my person, and was withdrawing the instrument, when, as if struck by a second thought, she resumed it, and so continued to regard me w...",0
id09553,There can be no doubt that the consciousness of the rapid increase of my superstition for why should I not so term it?,0


In [50]:
# Define X as the values from the "text" column. Define y as the "author" column.

X = df['text'].values
y = df['author']

In [52]:
# Convert to a dataset object using Dataset.from_tensor_slices()
ds = tf.data.Dataset.from_tensor_slices((X,y))
ds

<_TensorSliceDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [56]:
# Shuffle dataset
ds = ds.shuffle(buffer_size=len(ds),reshuffle_each_iteration=False) 

# Set the ratio of the train, validation, test split
split_train = .7
split_val =  .2
split_test =  1 -( split_train + split_val )

# Calculate the number of samples for training and validation data 
n_train_samples =  int(len(ds) * split_train)
n_val_samples = int(len(ds) * split_val)
n_test_samples = len(ds) -(n_train_samples + n_val_samples)

# Set the batch size
BATCH_SIZE =32


import math
# math.ceil will round up
# How many batches? 
n_train_batches = math.ceil(n_train_samples/BATCH_SIZE)
n_val_batches = math.ceil(n_val_samples/BATCH_SIZE)
n_test_batches = math.ceil(n_test_samples/BATCH_SIZE)

print(f"    - train:\t{n_train_samples} samples \t({n_train_batches} batches)")
print(f"    - val:  \t{n_val_samples} samples \t({n_val_batches} batches)")
print(f"    - test: \t{n_test_samples} samples \t({n_test_batches} batches)")

    - train:	11833 samples 	(370 batches)
    - val:  	3381 samples 	(106 batches)
    - test: 	1691 samples 	(53 batches)


In [57]:
# Use take and skip to define each set
train_ds = ds.take(n_train_samples).batch(batch_size=BATCH_SIZE)

# Skip over the training batches and take the validation batches
val_ds = ds.skip(n_train_samples).take(n_val_samples).batch(batch_size=BATCH_SIZE)

# Skipver the train and validation batches, the remaining are the test batches
test_ds = ds.skip(n_train_samples + n_val_samples).batch(batch_size=BATCH_SIZE)

In [58]:
# Create text Vectorization layer
sequence_vectorizer = tf.keras.layers.TextVectorization(
    standardize="lower_and_strip_punctuation",
    output_mode="int",
    output_sequence_length=100)


In [59]:
# Obtain the text from the training dataset using a lambda function
ds_texts = train_ds.map(lambda x ,y : x) 


In [60]:
# Train (adapt) the vectorization layer on the text data
sequence_vectorizer.adapt(ds_texts)

In [61]:
# Get vocabulary size
sequence_vectorizer.vocabulary_size()

20976

In [64]:
# Create a look-up dictionary that will allow you to look up the word associated with an integer. 

vocab = sequence_vectorizer.get_vocabulary()
int_to_str = {idx: word for idx, word in enumerate(vocab)}

int_to_str


{0: '',
 1: '[UNK]',
 2: 'the',
 3: 'of',
 4: 'and',
 5: 'to',
 6: 'i',
 7: 'a',
 8: 'in',
 9: 'was',
 10: 'that',
 11: 'my',
 12: 'it',
 13: 'he',
 14: 'had',
 15: 'with',
 16: 'his',
 17: 'as',
 18: 'for',
 19: 'but',
 20: 'which',
 21: 'not',
 22: 'me',
 23: 'at',
 24: 'from',
 25: 'by',
 26: 'on',
 27: 'this',
 28: 'is',
 29: 'her',
 30: 'be',
 31: 'were',
 32: 'have',
 33: 'you',
 34: 'all',
 35: 'an',
 36: 'we',
 37: 'or',
 38: 'no',
 39: 'when',
 40: 'him',
 41: 'one',
 42: 'so',
 43: 'they',
 44: 'been',
 45: 'could',
 46: 'would',
 47: 'she',
 48: 'there',
 49: 'upon',
 50: 'more',
 51: 'its',
 52: 'their',
 53: 'now',
 54: 'what',
 55: 'some',
 56: 'our',
 57: 'into',
 58: 'if',
 59: 'them',
 60: 'who',
 61: 'are',
 62: 'will',
 63: 'than',
 64: 'then',
 65: 'only',
 66: 'very',
 67: 'up',
 68: 'before',
 69: 'man',
 70: 'about',
 71: 'even',
 72: 'these',
 73: 'out',
 74: 'yet',
 75: 'your',
 76: 'time',
 77: 'did',
 78: 'any',
 79: 'old',
 80: 'said',
 81: 'might',
 82: 'li

In [65]:
# What word is associated with integer 1000?
int_to_str[1000]


'occasion'

In [66]:
# Obtain the vectorized result of the sample text: "Text Vectorization is an essential tool for converting text into a format that machine learning models can use."
sequence = sequence_vectorizer(["Text Vectorization is an essential tool for converting text into a format that machine learning models can use."])
sequence

<tf.Tensor: shape=(1, 100), dtype=int64, numpy=
array([[ 6345,     1,    28,    35,  6996, 13401,    18,     1,  6345,
           57,     7,     1,    10,   833,  3862, 16160,   122,   521,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0]], dtype=int64)>